In [1]:
import tqdm  # TODO: Agregar un poetry o requirements...
import json
import pandas as pd
import requests

SHEET_ID = '1b6h2RApBs2xbN6-eGVvxH68EALKDklvS91fb7d_IVz4'
SHEET_TAB = 'registros'

SHEET_URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_TAB}'

In [2]:
print(SHEET_URL)


https://docs.google.com/spreadsheets/d/1b6h2RApBs2xbN6-eGVvxH68EALKDklvS91fb7d_IVz4/gviz/tq?tqx=out:csv&sheet=registros


In [3]:
df_registros = pd.read_csv(SHEET_URL, dtype={'Padron': str})

In [4]:
df_registros.head()
df_registros[df_registros['Padron'].isna()]

,Timestamp,Padron,Carrera,Mapa
171,8/28/2022 21:49:05,NaN,electricista,"{""materias"":[{""id"":""CBC24"",""nota"":7},{""id"":""CB..."
190,8/28/2022 16:26:47,NaN,industrial,"{""materias"":[{""id"":""CBC24"",""nota"":9},{""id"":""CB..."
247,8/27/2022 13:52:37,NaN,industrial,"{""materias"":[{""id"":""CBC"",""nota"":0},{""id"":""81.0..."
543,8/24/2022 11:34:53,NaN,informatica,"{""materias"":[{""id"":""CBC28"",""nota"":7},{""id"":""CB..."
669,8/23/2022 8:53:16,NaN,quimica,"{""materias"":[{""id"":""CBC28"",""nota"":5},{""id"":""CB..."
...,...,...,...,...
4137,12/11/2019 22:45:46,NaN,sistemas,"{""materias"":[{""id"":""CBC"",""nota"":0}]}"
4148,11/27/2019 17:15:17,NaN,civil,"{""materias"":[{""id"":""CBC"",""nota"":0}]}"
4165,11/7/2019 9:59:22,NaN,informatica,"{""materias"":[{""id"":""CBC"",""nota"":0},{""id"":""62.0..."
4166,11/7/2019 9:58:07,NaN,informatica,"{""materias"":[{""id"":""CBC"",""nota"":0},{""id"":""62.0..."


In [5]:
# TODOs:
# - joinear con "usuarios" para obtener orientacion y tpp/tesis
# - ver si podemos rescatar "checkboxes" del json_mapa
# - ver si podemos migrar el legacy "materia_cuatri" con el nuevo "materia_cuatrimestre"
# - ver por qué hay padrones que no se toman (celda de arriba (teléfono para fede (será tema de formateo de dato numérico en gsheets?)))
dfr = df_registros
df_mapas = None
for idx in tqdm.tqdm(dfr.index):
    json_mapa = json.loads(dfr.loc[idx]['Mapa'])
    df_materias = pd.json_normalize(json_mapa['materias'])
    
    if 'optativas' not in json_mapa:
        df_optativas = pd.DataFrame()
    else:
        df_optativas = pd.json_normalize(json_mapa['optativas'])

    df_materias = df_materias.add_prefix('materia_')
    df_optativas = df_optativas.add_prefix('optativa_')

    df_map = pd.concat([df_materias, df_optativas]).reset_index(drop=True)
    df_map['Padron'] = dfr.loc[idx]['Padron']
    df_map['Carrera'] = dfr.loc[idx]['Carrera']
    df_map['Timestamp'] = dfr.loc[idx]['Timestamp']
    
    if df_mapas is None:
        df_mapas = df_map
    else:
        df_mapas = pd.concat([df_mapas, df_map])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4230/4230 [00:28<00:00, 149.81it/s]


In [6]:
df_mapas.head()

,materia_id,materia_nota,materia_cuatrimestre,Padron,Carrera,Timestamp,optativa_id,optativa_nombre,optativa_creditos,materia_cuatri
0,CBC24,5.0,NaN,103779,industrial,8/31/2022 19:57:20,NaN,NaN,NaN,NaN
1,CBC3,7.0,NaN,103779,industrial,8/31/2022 19:57:20,NaN,NaN,NaN,NaN
2,CBC5,7.0,NaN,103779,industrial,8/31/2022 19:57:20,NaN,NaN,NaN,NaN
3,CBC40,9.0,NaN,103779,industrial,8/31/2022 19:57:20,NaN,NaN,NaN,NaN
4,CBC,0.0,NaN,103779,industrial,8/31/2022 19:57:20,NaN,NaN,NaN,NaN


In [7]:
df_mapas.shape

(94335, 10)

In [19]:
# Gente que tiene pensado hacer Simu en el futuro...
df_mapas[df_mapas['materia_id'].isin(['75.26']) & (df_mapas['materia_cuatrimestre'] >= 2023)]['Padron']

43    101696
35    104956
41    104393
26     97009
42    107378
32     99248
36    105931
33    106295
40    107964
45    100972
36    102740
39    106004
41    105774
39    106440
47    105970
36    104351
31    108645
35       NaN
40    109071
34    102342
16    109555
40    103784
48    103775
42    103785
39    109131
34    104196
36     23456
Name: Padron, dtype: object